In [1]:
import numpy

from importlib import reload

from flow.core.params import SumoParams, EnvParams, \
    NetParams, InitialConfig, InFlows, SumoCarFollowingParams
from flow.core.params import VehicleParams
from flow.controllers import IDMController
from flow.envs.merge import MergePOEnv, ADDITIONAL_ENV_PARAMS
from flow.networks import MergeWithLightNetwork

from flow.core.experiment import Experiment

In [15]:
import flow
reload(flow)

from flow.networks import MergeWithLightNetwork
from flow.core.params import TrafficLightParams


def make_merge_flow_params(main_line_flow=2000,
                           on_ramp_flow=100,
                           cfm_controller=(IDMController,{"noise": 0.05})):
    
    vehicles = VehicleParams()
    vehicles.add(
        veh_id="human",
        acceleration_controller=cfm_controller,
        car_following_params=SumoCarFollowingParams(
            speed_mode="obey_safe_speed",
        ),
        num_vehicles=5)

    inflow = InFlows()
    inflow.add(
        veh_type="human",
        edge="inflow_highway",
        vehs_per_hour=main_line_flow,
        departLane="free",
        departSpeed=10)
    inflow.add(
        veh_type="human",
        edge="inflow_merge",
        vehs_per_hour=on_ramp_flow,
        departLane="free",
        departSpeed=7.5)

    # related to adding a light:
    tl_logic = TrafficLightParams(baseline=False)
    phases = [{
                "duration": "31",
                "minDur": "8",
                "maxDur": "45",
                "state": "GrGr"
            }, {
                "duration": "6",
                "minDur": "3",
                "maxDur": "6",
                "state": "yryr"
            }, {
                "duration": "31",
                "minDur": "8",
                "maxDur": "45",
                "state": "rGrG"
            }, {
                "duration": "6",
                "minDur": "3",
                "maxDur": "6",
                "state": "ryry"
            }]
#     tl_logic.add("inflow_merge", phases=phases, programID=1)
    tl_logic.add("inflow_merge",phases=phases)
    
    
    flow_params = dict(
        # name of the experiment
        exp_tag='mergewithlight-baseline',

        # name of the flow environment the experiment is running on
        env_name=MergePOEnv,

        # name of the network class the experiment is running on
        network=MergeWithLightNetwork,

        # simulator that is used by the experiment
        simulator='traci',

        # sumo-related parameters (see flow.core.params.SumoParams)
        sim=SumoParams(
            render=True,
            emission_path="./data/",
            sim_step=0.1,
            restart_instance=False,
        ),

        # environment related parameters (see flow.core.params.EnvParams)
        env=EnvParams(
            horizon=2000,
            additional_params=ADDITIONAL_ENV_PARAMS,
            sims_per_step=5,
            warmup_steps=0,
        ),

        # network-related parameters (see flow.core.params.NetParams and the
        # network's documentation or ADDITIONAL_NET_PARAMS component)
        net=NetParams(
            inflows=inflow,
            additional_params={
                "merge_length": 100,
                "pre_merge_length": 1000,
                "post_merge_length": 100,
                "merge_lanes": 1,
                "highway_lanes": 1,
                "speed_limit": 30,
            },
        ),

        # vehicles to be placed in the network at the start of a rollout (see
        # flow.core.params.VehicleParams)
        veh=vehicles,

        # parameters specifying the positioning of vehicles upon initialization/
        # reset (see flow.core.params.InitialConfig)
        initial=InitialConfig(
            spacing="uniform",
            perturbation=5.0,
        ),
        # traffic lights to be introduced to specific nodes (see
        # flow.core.params.TrafficLightParams)
        tls=tl_logic,
    )

    
    return flow_params

In [16]:
def run_sim(flow_params):
    exp = Experiment(flow_params, None)
    exp.run(1, convert_to_csv=True)
    print('Simulation finished.')
    
flow_params = make_merge_flow_params()
run_sim(flow_params)    

/Users/vanderbilt/Desktop/General_research_tools/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/Users/vanderbilt/Desktop/General_research_tools/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning
/Users/vanderbilt/Desktop/General_research_tools/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departLane in InFlows is deprecated, use depart_lane instead.
  PendingDeprecationWarning
/Users/vanderbilt/Desktop/General_research_tools/flow/flow/utils/flow_warnings.py:26: PendingDeprecationWarning: The attribute departSpeed in InFlows is deprecated, use depart_speed instead.
  PendingDeprecationWarning


Error during start: Traceback (most recent call last):
  File "/Users/vanderbilt/Desktop/General_research_tools/flow/flow/core/kernel/simulation/traci.py", line 251, in start_simulation
    traci_connection.setOrder(0)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/traci/connection.py", line 348, in setOrder
    self._sendExact()
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/traci/connection.py", line 99, in _sendExact
    raise FatalTraCIError("connection closed by SUMO")
traci.exceptions.FatalTraCIError: connection closed by SUMO

Error during teardown: [Errno 3] No such process
Error during start: Traceback (most recent call last):
  File "/Users/vanderbilt/Desktop/General_research_tools/flow/flow/core/kernel/simulation/traci.py", line 251, in start_simulation
    traci_connection.setOrder(0)
  File "/opt/anaconda3/envs/flow/lib/python3.6/site-packages/traci/connection.py", line 348, in setOrder
    self._sendExact()
  File "/opt/anaconda3/envs/flow/lib/

FatalTraCIError: connection closed by SUMO